In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import matplotlib.ticker as mticker
import time
import csv
import dateutil.parser, dateutil.tz as tz
import zoneinfo
from sklearn.preprocessing import MultiLabelBinarizer
import json

In [3]:
with open('../data/google_reviews/meta-Alaska.json', 'r') as f:
    data = [pd.json_normalize(json.loads(line)) for line in f]

# Concatenate the list of dataframes into a single dataframe
df = pd.concat(data, ignore_index=True)
display(df)

JSONDecodeError: Expecting value: line 2 column 1 (char 766)

In [ ]:
rating_fn = '../data/google_reviews/rating-Vermont.csv'
meta_fn = '../data/google_reviews/meta-Vermont.json'
# headers = ["user_id","parent_asin","rating","timestamp"]
rating_df = pd.read_csv(rating_fn, header=0)
display(rating_df)
with open(meta_fn, 'r') as f:
    data = [pd.json_normalize(json.loads(line)) for line in f]

meta_df = pd.concat(data, ignore_index=True)
# meta_df = pd.read_json(meta_fn)
# display(meta_df)

,business,user,rating,timestamp
0,0x89e02445cb9db457:0x37f42bff4edf7a43,104490523034805396984,5,1629987079369
1,0x89e02445cb9db457:0x37f42bff4edf7a43,112062684820633635547,5,1626442303453
2,0x89e02445cb9db457:0x37f42bff4edf7a43,110048311042721875910,5,1627319568225
3,0x89e02445cb9db457:0x37f42bff4edf7a43,106174372090234957142,5,1625223629975
4,0x89e02445cb9db457:0x37f42bff4edf7a43,106238734329148428463,5,1630520930973
...,...,...,...,...
841905,0x89e1c79c60aef325:0xbed8bfc41018f020,101031262113246567699,5,1503368927511
841906,0x89e1c79c60aef325:0xbed8bfc41018f020,109751562687532742649,5,1508992170442
841907,0x89e1c79c60aef325:0xbed8bfc41018f020,113511800890575185422,5,1566750149101
841908,0x89e1c79c60aef325:0xbed8bfc41018f020,107697312145259650604,5,1521208726615


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Royal Group,"Royal Group, 150 Woodstock Ave, Rutland, VT 05701",0x89e02445cb9db457:0x37f42bff4edf7a43,None,43.615537,-72.960497,"[Security system supplier, Fire protection equ...",4.9,148,None,"[[Thursday, 8AM–5PM], [Friday, 8AM–5PM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 8AM,"[0x89e0245c4c5c7d2d:0x97bca40a0223d767, 0x89e0...",https://www.google.com/maps/place//data=!4m2!3...
1,Foxglove Farm and Forest,"Foxglove Farm and Forest, 777 Delorm Rd, Leice...",0x4cb549e8877cf0d7:0xe8f003e6d73392ae,None,43.855743,-73.088180,"[Indoor lodging, Farm, Gift shop]",5.0,3,None,None,None,None,None,https://www.google.com/maps/place//data=!4m2!3...
2,Carr's Gifts,"Carr's Gifts, 21 Center St, Brandon, VT 05733",0x4cb54a301f3518f7:0x39af4eda1efb9117,None,43.799002,-73.088269,[Gift shop],4.9,13,None,"[[Wednesday, 10AM–5PM], [Thursday, 10AM–5PM], ...","{'Service options': ['In-store shopping', 'Del...",Closed ⋅ Opens 10AM Thu,"[0x4cb54b57c4f3123b:0x3d4e844c93c72568, 0x4cb5...",https://www.google.com/maps/place//data=!4m2!3...
3,Midas,"Midas, 207 US Route 4 E, Woodstock Ave, Rutlan...",0x89e024446398691f:0x4011d3a0f8636036,None,43.623157,-72.953303,"[Auto repair shop, Brake shop, Muffler shop, O...",4.3,48,None,"[[Wednesday, 7:30AM–5PM], [Thursday, 7:30AM–5P...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 7:30AM Thu,"[0x89e02489914d34f9:0x31610a8d324af00d, 0x89e0...",https://www.google.com/maps/place//data=!4m2!3...
4,Keyser Energy,"Keyser Energy, 77 Grove St Suite G102, Rutland...",0x89e0247d160d7263:0x32e4f01896e33f3b,None,43.611359,-72.981580,"[Heating oil supplier, Air conditioning contra...",4.7,68,None,"[[Wednesday, 8AM–4:30PM], [Thursday, 8AM–4:30P...",None,Closed ⋅ Opens 8AM Thu,"[0x89e0248f040ed50d:0x20f29a9817069602, 0x89e0...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11286,Ronald McDonald House Charities,"Ronald McDonald House Charities, 16 S Winooski...",0x4cca7af664d3c54b:0xf06a6e608b1f972,None,44.480157,-73.210569,[Non-profit organization],4.8,28,None,None,{'Accessibility': ['Wheelchair accessible entr...,NaN,"[0x4cca7afff6517e71:0x72429290ddd6f68d, 0x4cca...",https://www.google.com/maps/place//data=!4m2!3...
11287,Dollar General,"Dollar General, 35 Town Hwy 4, Alburg, VT 05440",0x4cc98bab87970361:0x39b72c846d9c2de0,Bargain retail chain selling a range of househ...,44.959080,-73.276770,"[Dollar store, Beauty supply store, Discount s...",3.7,178,$,"[[Saturday, 8AM–9PM], [Sunday, 8AM–9PM], [Mond...","{'Service options': ['In-store pickup', 'In-st...",NaN,"[0x4cca1aea68c1731d:0x4497a112aafbd4a8, 0x4cc9...",https://www.google.com/maps/place//data=!4m2!3...
11288,Alburgh RV Resort & Travel Sales,"Alburgh RV Resort & Travel Sales, 1 Alburg Rv ...",0x4cca209b5e50f0a7:0x4b2aa34d5902733d,None,44.965177,-73.247652,[Campground],3.6,18,None,None,None,NaN,None,https://www.google.com/maps/place//data=!4m2!3...
11289,MacLaomainn's Scottish Pub,"MacLaomainn's Scottish Pub, 52 S Main St, Ches...",0x89e1b111082d59bb:0x376c936a25aa35f,Classic neighborhood tavern in a former barn w...,43.262443,-72.594303,"[Pub, Restaurant]",4.5,397,$$,"[[Friday, 11:30AM–8PM], [Saturday, 11:30AM–8PM...","{'Service options': ['Outdoor seating', 'Curbs...",NaN,"[0x89e0965d0695d407:0x5d1977d381b207b7, 0x89e1...",https://www.google.com/maps/place//data=!4m2!3...


In [4]:
df = pd.merge(rating_df, meta_df, left_on=["business"], right_on=["gmap_id"])
df = df[["timestamp", "category"]]
df.dropna(inplace=True)
categories = df["category"].to_numpy()
unique_categories = list(set([item for sublist in categories for item in sublist]))
# for category in unique_categories:
#     df[category] = df['category'].apply(lambda x: 1 if category in x else 0)
# df.drop('category', axis=1, inplace=True)
display(df)

,timestamp,category
0,1629987079369,"[Security system supplier, Fire protection equ..."
1,1629987079369,"[Security system supplier, Fire protection equ..."
2,1626442303453,"[Security system supplier, Fire protection equ..."
3,1626442303453,"[Security system supplier, Fire protection equ..."
4,1627319568225,"[Security system supplier, Fire protection equ..."
...,...,...
843251,1503368927511,"[Orchard, Farm]"
843252,1508992170442,"[Orchard, Farm]"
843253,1566750149101,"[Orchard, Farm]"
843254,1521208726615,"[Orchard, Farm]"


In [5]:
mlb = MultiLabelBinarizer(sparse_output=True)

df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('category')),
                index=df.index,
                columns=mlb.classes_))
display(df)

# np_df = df.to_numpy()

,timestamp,3D printing service,ATM,ATV dealer,ATV rental service,ATV repair shop,Accountant,Accounting firm,Accounting school,Acupuncture clinic,...,Woodworking supply store,Work clothes store,Yamaha motorcycle dealer,Yarn store,Yoga retreat center,Yoga studio,Youth center,Youth hostel,Youth organization,Youth social services organization
0,1629987079369,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1629987079369,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1626442303453,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1626442303453,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1627319568225,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843251,1503368927511,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
843252,1508992170442,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
843253,1566750149101,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
843254,1521208726615,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def timestamp_to_timeseries(timestamps, start_time, end_time, bin_size):
    hist, _ = np.histogram(timestamps, bins=np.arange(start_time, end_time+bin_size, bin_size))
    return hist

def groupby(df, group_header='category_id', timestamp_header='timestamp'):
    np_df = df[[group_header, timestamp_header]].to_numpy()
    groups, groups_index = np.unique(np_df[:, 0], return_index=True)
    return np.split(np_df[:,1], groups_index[1:]), groups

def timeseries_to_csv(timeseries, group, csv_fn):
    timeseries_group = [group]
    timeseries_group.extend(list(timeseries))
    with open(csv_fn, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(timeseries_group)

In [6]:
# df['time']=pd.to_datetime(df['timestamp'],unit='s')+datetime.timedelta(hours=8)
start_time = datetime.datetime(2007, 1, 1, 00, 00).timestamp()
end_time = datetime.datetime(2021, 9, 30, 00, 00).timestamp()
bin_size = 60*60*24*7 # in seconds
# csv_fn = '../data/google_reviews_vermont.csv'
timestamp_header = 'timestamp'
group_header = 'category'
print(start_time, end_time)

# timeseries_to_csv(np.arange(start_time, end_time+bin_size, bin_size), group_header, csv_fn)
# timestamps_grouped, groups= groupby(df, group_header, timestamp_header)
# for timestamps, group in zip(timestamps_grouped, groups):
#     timeseries = timestamp_to_timeseries(timestamps, start_time, end_time, bin_size)
#     timeseries_to_csv(timeseries, group, csv_fn)

# 1602133857705

1167638400.0 1632985200.0


In [11]:
# states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "District_of_Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New_Hampshire", "New_Jersey", "New_Mexico", "New_York", "North_Carolina", "North_Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode_Island", "South_Carolina", "South_Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West_Virginia", "Wisconsin", "Wyoming"]
states = ["West_Virginia", "Wisconsin", "Wyoming"]
for state in states:
    start_count_time = time.time()
    rating_fn = f'../data/google_reviews/rating-{state}.csv'
    meta_fn = f'../data/google_reviews/meta-{state}.json'
    csv_fn = f'../data/google_reviews/timeseries_{state}.csv'
    # headers = ["user_id","parent_asin","rating","timestamp"]
    rating_df = pd.read_csv(rating_fn, header=0)
    meta_df = pd.read_json(meta_fn)
    print('data load time', time.time() - start_count_time)
    df = pd.merge(rating_df, meta_df, left_on=["business"], right_on=["gmap_id"])
    df = df[["timestamp", "category"]]
    df.dropna(inplace=True)

    mlb = MultiLabelBinarizer(sparse_output=True)

    df = df.join(
                pd.DataFrame.sparse.from_spmatrix(
                    mlb.fit_transform(df.pop('category')),
                    index=df.index,
                    columns=mlb.classes_))

    timeseries_to_csv(np.arange(start_time, end_time+bin_size, bin_size), group_header, csv_fn)
    for i, category in enumerate(df.columns[1:]):
        np_df = df[['timestamp', category]].to_numpy()
        category_np = np_df[np_df[:,1]==1, 0] // 1000
        hist = timestamp_to_timeseries(category_np, start_time, end_time, bin_size)
        timeseries_to_csv(hist, category, csv_fn)
    print(f'{state} took {time.time()-start_count_time} seconds')

data load time 3.2491049766540527
West_Virginia took 34.037235260009766 seconds
data load time 7.966877698898315
Wisconsin took 219.531720161438 seconds
data load time 1.0492596626281738
Wyoming took 21.37627339363098 seconds


In [19]:
timeseries_fn = f'../data/google_reviews_weekly.csv'
timeseries_df = pd.read_csv(timeseries_fn, header=0)
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "District_of_Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New_Hampshire", "New_Jersey", "New_Mexico", "New_York", "North_Carolina", "North_Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode_Island", "South_Carolina", "South_Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West_Virginia", "Wisconsin", "Wyoming"]
for state in states:
    state_csv = f'../data/google_reviews/timeseries_{state}.csv'
    timeseries_state_df = pd.read_csv(state_csv, header=0)
    timeseries_df = pd.concat([timeseries_df, timeseries_state_df], axis=0)
    timeseries_df = timeseries_df.groupby('category', as_index=False).sum()
timeseries_df.to_csv(timeseries_fn, index=False)

C:\Users\Coen\AppData\Local\Temp\ipykernel_53708\941239912.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  timeseries_df = timeseries_df.groupby('category', as_index=False).sum()
C:\Users\Coen\AppData\Local\Temp\ipykernel_53708\941239912.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  timeseries_df = timeseries_df.groupby('category', as_index=False).sum()
C:\Users\Coen\AppData\Local\Temp\ipykernel_53708\941239912.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inser

In [20]:
timeseries_fn = f'../data/google_reviews_weekly.csv'
timeseries_df = pd.read_csv(timeseries_fn, header=0)
display(timeseries_df)

,category,1167638400.0,1168243200.0,1168848000.0,1169452800.0,1170057600.0,1170662400.0,1171267200.0,1171872000.0,1172476800.0,...,1627286400.0,1627891200.0,1628496000.0,1629100800.0,1629705600.0,1630310400.0,1630915200.0,1631520000.0,1632124800.0,1632729600.0
3D printing service,0,0,0,0,0,0,0,0,0,0,...,5,0,0,0,1,0,0,0,0,0
ATM,0,0,0,0,0,0,0,0,0,0,...,949,646,537,296,146,19,0,0,0,0
ATV dealer,0,0,0,0,0,0,0,0,0,0,...,42,26,21,15,18,0,0,0,0,0
ATV rental service,0,0,0,0,0,0,0,0,0,0,...,40,21,18,8,4,0,0,0,0,0
ATV repair shop,0,0,0,0,0,0,0,0,0,0,...,61,42,25,10,15,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Youth organization,0,0,0,0,0,0,0,0,0,0,...,52,37,27,13,11,0,0,0,0,0
Youth social services organization,0,0,0,0,0,0,0,0,0,0,...,11,2,2,0,0,0,0,0,0,0
Yucatan restaurant,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zoo,0,0,0,0,0,0,0,0,0,0,...,34,29,8,3,5,0,0,0,0,0
